# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis , we have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. now we will be performing more interactive visual analytics using `Folium`.


## Objectives


*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, we should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [52]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [53]:
import folium
import pandas as pd

In [54]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [55]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, we can take a look at what are the coordinates for each site.


In [56]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give us any intuitive insights about where are those launch sites. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [57]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [58]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Now, let's add a circle for each launch site in data frame `launch_sites`


In [59]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Iterate over launch sites and add circles and markers
for index, row in launch_sites_df.iterrows():
    # Create a circle for each launch site with a radius of 1000 meters
    circle = folium.Circle([row['Lat'], row['Long']], radius=1000, color='#000000', fill=True).add_child(folium.Popup(row['Launch Site']))
    site_map.add_child(circle)
    
    # Create a marker for each launch site with a popup label showing its name
    marker = folium.Marker(
        [row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )
    ).add_child(folium.Popup(row['Launch Site']))  # Add a popup label with the launch site name
    site_map.add_child(marker)
# Display the map
site_map


***1.Proximity to the Equator line:***
By observing the map, we can see that not all launch sites are in proximity to the Equator line. Some launch sites, such as those in the United States and Europe, are located at higher latitudes, farther away from the Equator. However, some launch sites, such as those in Florida (e.g., Kennedy Space Center) and French Guiana (e.g., Guiana Space Centre), are closer to the Equator.

***2.Proximity to the coast:***
Most launch sites appear to be in close proximity to the coast. Launch sites often benefit from being located near the coast due to safety reasons (launching over water minimizes risks to populated areas) and logistical advantages (ease of transporting rockets and equipment by sea). Many launch sites, such as Kennedy Space Center, Cape Canaveral Space Launch Complex, and Vandenberg Space Force Base, are situated directly on the coast. However, there are exceptions, such as Baikonur Cosmodrome in Kazakhstan, which is inland.

->Launch sites closer to the Equator can take advantage of the Earth's rotational speed, which is higher near the Equator. This additional rotational velocity assists in achieving greater payload capacity or reducing the fuel required for achieving certain orbits. Hence, some launch providers prefer to locate their launch sites near the Equator.

->Launch sites near the coast provide safety benefits, as launching over water minimizes risks to populated areas in case of accidents or malfunctions during launch. Additionally, proximity to the coast facilitates transportation of large rockets and equipment by sea, reducing logistical challenges.

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [60]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [61]:
marker_cluster = MarkerCluster()


In [62]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']

# Create a new column in launch_sites dataframe called marker_color to store the marker colors based on the class value
spacex_df['marker_color'] = ''

# Define a function to set marker color based on class value
def set_marker_color(row):
    if row['class'] == 1:
        return 'green'  # Success: Green color
    else:
        return 'red'  # Failed: Red color

# Apply the function to set marker colors
spacex_df['marker_color'] = spacex_df.apply(set_marker_color, axis=1)

# For each launch result in spacex_df data frame, add a folium.Marker to marker_cluster
for index, row in spacex_df.iterrows():
    marker = folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color='white', icon_color=row['marker_color'])
    ).add_to(marker_cluster).add_child(folium.Popup(row['Launch Site']))

# Add marker cluster to the map
site_map.add_child(marker_cluster)

site_map.add_child(marker)

site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [63]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [64]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [65]:
# find coordinate of the closet coastline
distance_coastline = calculate_distance(28.563197	, -80.576820 , 28.56333 , -80.56799)
distance_coastline

0.8627473698047621

In [66]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_marker = folium.Marker(
    [28.56333,-80.56799],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
site_map.add_child(distance_marker)
site_map

In [67]:
# Define the coordinates of the launch site and the coastline
launch_site_lat = 28.563197
launch_site_lon = -80.576820
coastline_lat = 28.56333
coastline_lon = -80.56799

# Create a list of coordinates for the PolyLine
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]

# Create a PolyLine object
polyline = folium.PolyLine(locations=coordinates, weight=1)

# Add the PolyLine to the map
site_map.add_child(polyline)

# Display the map
site_map


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [68]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
import folium
from folium.features import DivIcon

# Function to calculate distance between two points using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    import math

    # Convert latitude and longitude from decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371 # Radius of earth in kilometers
    return c * r

# Create a map
site_map = folium.Map(location=[28.563197, -80.576820], zoom_start=10)

# Define the coordinates of the launch site and the railway
launch_site_lat = 28.563197
launch_site_lon = -80.576820
railway_lat = 28.57207
railway_lon = -80.58527

# Calculate the distance between the launch site and the railway
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

# Create a marker for the railway point with the distance displayed
distance_marker = folium.Marker(
    [railway_lat, railway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance_railway),
    )
)

# Add the distance marker to the map
site_map.add_child(distance_marker)

# Create a list of coordinates for the PolyLine
coordinates = [[launch_site_lat, launch_site_lon], [railway_lat, railway_lon]]

# Create a PolyLine object
polyline = folium.PolyLine(locations=coordinates, weight=1)

# Add the PolyLine to the map
site_map.add_child(polyline)

# Display the map
site_map


In [69]:
# Given coordinates for the launch site and the highway
launch_site_lat = 28.563197
launch_site_lon = -80.576820
highway_lat = 28.56425
highway_lon = -80.57092

# Calculating the distance between the launch site and the highway
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# Creating a marker for the highway point with the distance displayed
distance_marker_highway = folium.Marker(
    [highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance_highway),
    )
)

# Adding the distance marker to the map
site_map.add_child(distance_marker_highway)

# Creating a list of coordinates for the PolyLine
coordinates_highway = [[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]]

# Creating a PolyLine object
polyline_highway = folium.PolyLine(locations=coordinates_highway, weight=1)

# Adding the PolyLine to the map
site_map.add_child(polyline_highway)

# Displaying the map
site_map


In [70]:
# Given coordinates for the launch site and the city
launch_site_lat = 28.563197
launch_site_lon = -80.576820
city_lat = 28.07852
city_lon = -80.62454

# Calculating the distance between the launch site and the city
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Creating a marker for the city point with the distance displayed
distance_marker_city = folium.Marker(
    [city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance_city),
    )
)

# Adding the distance marker to the map
site_map.add_child(distance_marker_city)

# Creating a list of coordinates for the PolyLine
coordinates_city = [[launch_site_lat, launch_site_lon], [city_lat, city_lon]]

# Creating a PolyLine object
polyline_city = folium.PolyLine(locations=coordinates_city, weight=1)

# Adding the PolyLine to the map
site_map.add_child(polyline_city)

# Displaying the map
site_map


In [71]:
# Given coordinates for the launch site and the coastline
launch_site_lat = 28.563197
launch_site_lon = -80.576820
coastline_lat = 28.56333
coastline_lon = -80.56799

# Calculating the distance between the launch site and the coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Creating a marker for the coastline point with the distance displayed
distance_marker_coastline = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance_coastline),
    )
)

# Adding the distance marker to the map
site_map.add_child(distance_marker_coastline)

# Creating a list of coordinates for the PolyLine
coordinates_coastline = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]

# Creating a PolyLine object
polyline_coastline = folium.PolyLine(locations=coordinates_coastline, weight=1)

# Adding the PolyLine to the map
site_map.add_child(polyline_coastline)

# Displaying the map
site_map
